In [51]:
import os
import pandas as pd
import json
import glob
import sklearn
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import  classification_report
from sklearn.metrics import confusion_matrix

In [6]:
city_list=os.listdir("C:\\Users\\Abi\\Desktop\\ML_project\\weather-20200731T142422Z-001\\weather")
file_path="C:\\Users\\Abi\\Desktop\\ML_project\\weather-20200731T142422Z-001\\weather\\"
weather_list=[]
for p in city_list:
    json_list=os.listdir(file_path+p)
    s=json_list.index('2016-1.json')
    actual_list=json_list[s:]
    for i in actual_list:
        fp=open(file_path+p+"\\"+i,"r")
        d=fp.read()
        data=json.loads(d)
        feature_list=['windspeedKmph','winddirDegree','weatherCode','precipMM','visibility','pressure','cloudcover','DewPointF','WindGustKmph',         'tempF','WindChillF','humidity','time']
        
        for i in data["data"]["weather"]:
            for j in range(24):
                weather_per_hr=dict()
                for k in feature_list:
                    weather_per_hr[k]=i["hourly"][j][k]
                weather_per_hr['date']=i['date']
                weather_per_hr['airport']=data["data"]["request"][0]["query"]
                weather_list.append(weather_per_hr)
        fp.close()
df_weather=pd.DataFrame(weather_list)

df_weather.to_csv("weather_features.csv")

In [7]:
df_weather["air"]=""

In [8]:
df_weather["air"]=[x[0:3] for x in df_weather["airport"]]

In [9]:
df_weather["int_time"]=0

In [10]:
df_weather["int_time"]=[int(x) for x in df_weather["time"]]


In [11]:
df_weather.shape

(263160, 17)

In [12]:
df_weather

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,time,date,airport,air,int_time
0,11,318,176,0.1,10,1023,100,47,17,49,46,91,0,2016-01-01,"ATL, Hartsfield-Jackson Atlanta International ...",ATL,0
1,13,317,176,0.0,10,1023,100,44,22,46,42,92,100,2016-01-01,"ATL, Hartsfield-Jackson Atlanta International ...",ATL,100
2,14,315,122,0.0,10,1023,100,41,26,43,38,92,200,2016-01-01,"ATL, Hartsfield-Jackson Atlanta International ...",ATL,200
3,16,314,122,0.0,10,1023,100,38,30,40,33,93,300,2016-01-01,"ATL, Hartsfield-Jackson Atlanta International ...",ATL,300
4,17,314,122,0.0,10,1023,100,38,30,40,33,93,400,2016-01-01,"ATL, Hartsfield-Jackson Atlanta International ...",ATL,400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263155,26,296,113,0.0,10,1014,0,57,27,67,67,70,1900,2017-09-30,"SFO, San Francisco International Airport, Unit...",SFO,1900
263156,26,298,113,0.0,10,1014,0,58,27,67,67,71,2000,2017-09-30,"SFO, San Francisco International Airport, Unit...",SFO,2000
263157,26,300,113,0.0,10,1014,0,58,27,67,67,73,2100,2017-09-30,"SFO, San Francisco International Airport, Unit...",SFO,2100
263158,23,302,113,0.0,10,1014,0,58,25,66,66,74,2200,2017-09-30,"SFO, San Francisco International Airport, Unit...",SFO,2200


In [13]:
files=glob.glob('C:\\Users\\Abi\\Desktop\\ML_project\\Flight Data-20200805T074725Z-001\\Flight Data\**\*',recursive=True)
df_flight=pd.DataFrame()
for i in files:
    try:
        i.index(".csv")
    except:
        continue
    d=pd.read_csv(i)
    d=d[['FlightDate','Quarter','Year','Month'
    ,'DayofMonth','DepTime','DepDel15','CRSDepTime','DepDelayMinutes','OriginAirportID','DestAirportID','ArrTime','CRSArrTime','ArrDel15',           'ArrDelayMinutes','Origin','Dest']]
    df_flight=pd.concat([df_flight,d])

    

In [14]:
airports=['ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO']
df_flight=df_flight[df_flight["Dest"].isin(airports)]
df_flight=df_flight[df_flight["Origin"].isin(airports)]





In [15]:
df_flight=df_flight.dropna()

In [16]:
df_flight["round_time"]=0

In [17]:
#to calculate the round-off value for the departure time
temp=[]
for x in df_flight["CRSDepTime"]: 
    if(x%100<30):
        temp.append((int(x/100)*100)%2400)
    else:
        temp.append(((int(x/100)+1)*100)%2400)
df_flight["round_time"]=temp

In [18]:
df_flight.shape

(1851436, 18)

In [19]:
df_flight

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,Origin,Dest,round_time
57,2016-01-01,1,2016,1,1,741.0,0.0,745,0.0,14747,12478,1610.0,1602,0.0,8.0,SEA,JFK,800
58,2016-01-02,1,2016,1,2,737.0,0.0,745,0.0,14747,12478,1613.0,1602,0.0,11.0,SEA,JFK,800
59,2016-01-03,1,2016,1,3,743.0,0.0,745,0.0,14747,12478,1547.0,1602,0.0,0.0,SEA,JFK,800
60,2016-01-04,1,2016,1,4,737.0,0.0,745,0.0,14747,12478,1551.0,1602,0.0,0.0,SEA,JFK,800
61,2016-01-05,1,2016,1,5,708.0,0.0,710,0.0,14747,12478,1524.0,1527,0.0,0.0,SEA,JFK,700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458684,2017-09-25,3,2017,9,25,2038.0,0.0,2045,0.0,14771,12478,449.0,522,0.0,0.0,SFO,JFK,2100
458687,2017-09-25,3,2017,9,25,1200.0,0.0,1205,0.0,11618,13204,1428.0,1449,0.0,0.0,EWR,MCO,1200
458688,2017-09-25,3,2017,9,25,1143.0,0.0,1141,2.0,13204,11618,1409.0,1417,0.0,0.0,MCO,EWR,1200
458695,2017-09-25,3,2017,9,25,2157.0,0.0,2159,0.0,12889,12478,540.0,557,0.0,0.0,LAS,JFK,2200


In [20]:
#merging weather dataframe and flight dataframe
result=pd.merge(left=df_weather,right=df_flight,left_on=["date","int_time","air"],right_on=["FlightDate","round_time","Origin"])

In [22]:
result=result.dropna()

In [24]:
target=result["ArrDel15"]

In [25]:
features=result.copy()
features=features.drop(["ArrDel15"],axis=1)

In [27]:
features=features.drop(["date","airport","air","FlightDate","Origin","Dest","ArrDelayMinutes"],axis=1)

In [28]:
feature_selt=pd.DataFrame(SelectKBest(f_classif,k=20).fit_transform(features,target))

In [29]:
#feature set
train_feature=feature_selt.head(int(0.8*len(features)))
test_feature=feature_selt.tail(int(0.2*len(features)))

In [30]:
#target set
train_target=target.head(int(0.8*len(target)))
test_target=target.tail(int(0.2*len(features)))

In [31]:
#logistic regression classification
LogReg=LogisticRegression()
LogReg.fit(train_feature,train_target)
logit_predict=LogReg.predict(test_feature)

In [58]:
count_zero=0
for i in logit_predict:
    if(i==0):
        count_zero+=1
print("no. of ones: ",len(logit_predict)-count_zero)
print("no. of zeros: ",count_zero)


no. of ones:  57010
no. of zeros:  313277


In [34]:
#computes the accuracy of  logistic regression
count=0
tes_tar=list(test_target)
logit=list(logit_predict)
for i in range(len(test_target)):
    if(tes_tar[i]==logit[i]):
        count+=1
print(count/len(test_target))      


0.9153737506312671


In [62]:
print("Accuracy:",metrics.accuracy_score(test_target,logit_predict))

Accuracy: 0.9153737506312671


In [35]:
#calculating the average precision for the logistic model
average_precision = average_precision_score(test_target,logit_predict)
print("precision: ",average_precision)

0.6776567461106002


In [50]:
print(classification_report(test_target,logit_predict))

precision    recall  f1-score   support

         0.0       0.92      0.98      0.95    291769
         1.0       0.91      0.66      0.77     78518

    accuracy                           0.92    370287
   macro avg       0.91      0.82      0.86    370287
weighted avg       0.92      0.92      0.91    370287



In [36]:
 #decision tree classifier
 dtc=DecisionTreeClassifier()
 dtc=dtc.fit(train_feature,train_target)
 tree_pred=dtc.predict(test_feature)
 

In [59]:
count_zero=0
for i in tree_pred:
    if(i==0):
        count_zero+=1
print("no. of ones: ",len(logit_predict)-count_zero)
print("no. of zeros: ",count_zero)

no. of ones:  77264
no. of zeros:  293023


In [37]:
#calculating accuracy
print("Accuracy:",metrics.accuracy_score(test_target,tree_pred))

Accuracy: 0.9555749999324848


In [38]:
#calculating the average precision for the decision tree model
average_precision = average_precision_score(test_target,tree_pred)
print("precision",average_precision)

0.8239156309020073


In [53]:
print(classification_report(test_target,tree_pred))

precision    recall  f1-score   support

         0.0       0.97      0.97      0.97    291769
         1.0       0.90      0.89      0.89     78518

    accuracy                           0.96    370287
   macro avg       0.94      0.93      0.93    370287
weighted avg       0.96      0.96      0.96    370287



In [45]:
#xgboost
xgb = GradientBoostingClassifier(random_state=0)
xgb.fit(train_feature,train_target)

GradientBoostingClassifier(random_state=0)

In [46]:
xgb_pred=xgb.predict(test_feature)

In [60]:
count_zero=0
for i in xgb_pred:
    if(i==0):
        count_zero+=1
print("no. of ones: ",len(logit_predict)-count_zero)
print("no. of zeros: ",count_zero)

no. of ones:  58074
no. of zeros:  312213


In [47]:
#accuracy
print("Accuracy:",metrics.accuracy_score(test_target,xgb_pred))

Accuracy: 0.9216337597593218


In [54]:
#precision score
print("average precision:",average_precision_score(test_target,tree_pred))

average precision: 0.8239156309020073


In [55]:
print(classification_report(test_target,xgb_pred))

precision    recall  f1-score   support

         0.0       0.92      0.99      0.95    291769
         1.0       0.93      0.69      0.79     78518

    accuracy                           0.92    370287
   macro avg       0.92      0.84      0.87    370287
weighted avg       0.92      0.92      0.92    370287



In [41]:
#extra tree classifier
etc=ExtraTreesClassifier(n_estimators=100,random_state=0)
etc.fit(train_feature,train_target)
extratree_pred=etc.predict(test_feature)


In [61]:
count_zero=0
for i in extratree_pred:
    if(i==0):
        count_zero+=1
print("no. of ones: ",len(logit_predict)-count_zero)
print("no. of zeros: ",count_zero)

no. of ones:  61015
no. of zeros:  309272


In [42]:
#accuracy
print("Accuracy:",metrics.accuracy_score(test_target,extratree_pred))

Accuracy: 0.9243424694898822


In [56]:
#precision score
print("average precision:",average_precision_score(test_target,   tree_pred))

average precision: 0.8239156309020073


In [57]:
print(classification_report(test_target,extratree_pred))

precision    recall  f1-score   support

         0.0       0.93      0.98      0.95    291769
         1.0       0.91      0.71      0.80     78518

    accuracy                           0.92    370287
   macro avg       0.92      0.85      0.88    370287
weighted avg       0.92      0.92      0.92    370287

